<a href="https://colab.research.google.com/github/jpbeaud/dbscan/blob/main/client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transformation fichiers CSV en DataFrame

In [2]:
import pandas as pd
import numpy as np
import time

def string_to_number(s):
  return int(''.join(str(ord(char)) for char in s))



chemin = "/content/sample_data/"
debut = time.time()
print("Début à", debut)



fichier_DH = pd.read_csv(chemin+"extrac_DH2.csv", sep=",", dtype=str,skipinitialspace=True)

fichier_DH = fichier_DH.replace("", np.nan)
fichier_DH = fichier_DH.fillna(0)

fichier_DH = fichier_DH.astype(str)
col_ref = ['id_personne']
references_DH = fichier_DH[col_ref]



fichier_DH.sort_values(by='id_personne')

fichier_DH2 = fichier_DH
fichier_DH2 = fichier_DH2.drop('id_personne', axis='columns')

fichier_DH2= fichier_DH2.astype(str)
fichier_DH2 = fichier_DH2.map(string_to_number)



#fichier_DH.set_index('id_personne', inplace=True) # inplace=True --> on ne crée pas un nouveau DF




fichier_SIO = pd.read_csv(chemin+"extrac_SIO2.csv", sep=",", dtype=str,skipinitialspace=True)
col_ref = ['client_reference']

references_SIO= fichier_SIO[col_ref]
ref_cli = references_SIO.to_numpy()

fichier_SIO = fichier_SIO.replace("", np.nan)
fichier_SIO = fichier_SIO.fillna(0)



fichier_SIO.sort_values(by='client_reference')
fichier_SIO = fichier_SIO.astype(str)
fichier_SIO = fichier_SIO.map(string_to_number)


fichier_SIO = fichier_SIO.drop('client_reference',axis='columns')
#print ("fichier SIO format :")
#print(fichier_SIO.head())
# X = fichier_DH.to_numpy()



Début à 1731353384.4614975


Nombre d'enregistrements différents (hors Client-reference)

In [3]:
# prompt: comment calculer le nombre d'enregistrements différents dans fichier_DH2

# Calculate the number of unique records in fichier_DH2
unique_records_count = len(fichier_DH2.drop_duplicates())

print(f"Number of unique records in fichier_DH2: {unique_records_count}")

# Calculate the number of unique records in fichier_SIO
unique_records_count = len(fichier_SIO.drop_duplicates())

print(f"Number of unique records in fichier_SIO: {unique_records_count}")

Number of unique records in fichier_DH2: 14725
Number of unique records in fichier_SIO: 17084


Appel à DBSCAN

In [4]:
X = fichier_DH2
#X.set_index('id_personne', inplace=True) # inplace=True --> on ne crée pas un nouveau DF

# print(X.dtype)
X = X.apply(pd.to_numeric)

import matplotlib.pyplot as plt
# pltcatter(X[:, 3], X[:, 4])


#plt.scatter(x="pcs" , y="cosop", c = 'r', data=X)
#plt.show()

from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

# création du jeu de test

#X = StandardScaler().fit_transform(X)
#print ("fichier DH2 après scaler :")
#print(X[:5])

from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
min_groupe = 2
db = DBSCAN(eps=0.000000000000001,min_samples=min_groupe).fit(X)
labels_DH = db.labels_

n_clusters_ = len(set(labels_DH)) - (1 if -1 in labels_DH else 0)
n_noise_ = list(labels_DH).count(-1)

print("Estimated number of clusters DH: %d" % n_clusters_)
print("Estimated number of noise points DH: %d" % n_noise_)


Y = fichier_SIO


#Y = StandardScaler().fit_transform(Y)
#print ("fichier SIO après scaler :")
#print(Y[:5])


# Y = Y.apply(pd.to_numeric)
db2 = DBSCAN(eps=0.00000000000001, min_samples=min_groupe).fit(Y)
labels_SIO = db2.labels_



n_clusters_ = len(set(labels_SIO)) - (1 if -1 in labels_SIO else 0)
n_noise_ = list(labels_SIO).count(-1)

print("Estimated number of clusters SIO: %d" % n_clusters_)
print("Estimated number of noise points SIO: %d" % n_noise_)
print("____________Résumé____________")
difference = labels_DH - labels_SIO
print("Données en écarts = ", difference)
print(labels_DH)
print(labels_SIO)
n_DH = len(labels_DH)
n_SIO=len(labels_SIO)
n_dif = len(difference)
print("Longueur dif = ", n_dif)
fin = False
i=-1

nb_ko = 0

# Créer un tableau vide avec une dimension initiale
empty_array = np.empty((0, 3))  # Par exemple, un tableau vide avec 3 colonnes



Estimated number of clusters DH: 4362
Estimated number of noise points DH: 10231
Estimated number of clusters SIO: 4685
Estimated number of noise points SIO: 11871
____________Résumé____________
Données en écarts =  [  0   0   0 ... -12   1 -12]
[ 0  1  2 ...  7 29  7]
[ 0  1  2 ... 19 28 19]
Longueur dif =  100000


On sélectionne les enregistrements qui ont 0 comme label

In [ ]:
labels_DH_zero = [x for x in labels_DH if x ==0]
print("longueur DH label 0 =", len(labels_DH_zero))
labels_SIO_zero = [x for x in labels_SIO if x ==0]
print("longueur SIO label 0 =", len(labels_SIO_zero))


longueur DH label 0 = 204
longueur SIO label 0 = 190


In [ ]:
# prompt: transform labels_DH into a dataframe

#print("DH.38=", X[38], "SIO.38=", Y[38])
#print("DH.0=", X[0], "SIO.0=", Y[0])

# Assuming 'labels_DH' is already defined from the previous code
labels_DH_df = pd.DataFrame({'labels_DH': labels_DH})
labels_SIO_df = pd.DataFrame({'labels_SIO': labels_SIO})

labels_DH_0 = labels_DH_df[labels_DH_df['labels_DH'] == 0]
print(labels_DH_0.head())
labels_SIO_0 = labels_SIO_df[labels_SIO_df['labels_SIO'] == 0]
print(labels_SIO_0.head())
print("______________________________________________________")
#print(labels_DH[38], " SIO=", labels_SIO[38])
#print("Référence SIO", references_SIO.iloc[38], ". Référence DH = ", references_DH.iloc[38])




    labels_DH
0           0
12          0
29          0
40          0
53          0
    labels_SIO
0            0
12           0
29           0
40           0
53           0
______________________________________________________


Extraction des index en décalage

In [ ]:
# prompt: extraire les index qui dont dans labels_DH_0 mais pas dans labels_SIO_0

# Find indices present in labels_DH_0 but not in labels_SIO_0
dh_indices_in_0 = labels_DH_0.index.tolist()
sio_indices_in_0 = labels_SIO_0.index.tolist()

indices_in_sio_not_dh = list(set(sio_indices_in_0) - set(dh_indices_in_0))

print("Indices in labels_DH_0 but not in labels_SIO_0:")
print(indices_in_sio_not_dh)
len(indices_in_sio_not_dh)

Indices in labels_DH_0 but not in labels_SIO_0:
[]


0

Test A

In [ ]:
print(sio_indices_in_0)
print("Label DH SIO du 0ème ", labels_DH[0])
print("Label DH du 12 ème ",labels_DH[12])
print("Label DH du 1er enr. = ", labels_DH[sio_indices_in_0[0]], "pos=", sio_indices_in_0[0])
print("Label SIO du 2èmz enr. = ", labels_SIO[sio_indices_in_0[1]], "pos=", sio_indices_in_0[1])

[0, 12, 29, 40, 53, 57, 64, 73, 75, 101, 105, 107, 110, 123, 124, 138, 149, 151, 179, 197, 270, 272, 279, 295, 307, 397, 575, 690, 704, 750, 786, 962, 1290, 1636, 2706, 4651, 9562, 10025, 10163, 10244, 10539, 10573, 10584, 10745, 11465, 11641, 11717, 12244, 12402, 13532, 14286, 15487, 15851, 17717, 19102, 19423, 19549, 21497, 22741, 23110, 24402, 26018, 27146, 27239, 27528, 27722, 27753, 28063, 28103, 29528, 30934, 31472, 33327, 33978, 34811, 36024, 36163, 37320, 37350, 38293, 38474, 38481, 38517, 38519, 38521, 38526, 38565, 38577, 38582, 38593, 38594, 38600, 38622, 38858, 39038, 39528, 39635, 39695, 40103, 40114, 41699, 42786, 43290, 46375, 46488, 46631, 46632, 47441, 47491, 47930, 47948, 48123, 48394, 48416, 49071, 50523, 51390, 51624, 52073, 52556, 54493, 55193, 58831, 59244, 59600, 61637, 61788, 62659, 63373, 64966, 65253, 65932, 66241, 67947, 72701, 73332, 75942, 75951, 76128, 76134, 76153, 76175, 76179, 76191, 76203, 76209, 76238, 76242, 76246, 76250, 76263, 76273, 76417, 76479, 

Nombre d'enregistrements en décallage

In [7]:
#i0 = indices_in_sio_not_dh[0]
#print("Donnée DH avec indice ", i0)
#print(X.iloc[i0])
#print("Donnée SIO avec indice ", i0)
#print(Y.iloc[i0])
#print("Référence SIO", references_SIO.iloc[i0], ". Référence DH = ", references_DH.iloc[i0])
labels_DH_df = pd.DataFrame({'labels_DH': labels_DH})
labels_SIO_df = pd.DataFrame({'labels_SIO': labels_SIO})

liste_ref_KO = []
for num_label in range(10):
  print("num_label=", num_label)
  print("-------")
  labels_SIO_num_label = labels_SIO_df[labels_SIO_df['labels_SIO'] == num_label]
  sio_indices_in_num_label = labels_SIO_num_label.index.tolist()
  #print("Nombre d'enr = ", len(sio_indices_in_i))
  k = labels_DH[sio_indices_in_num_label[num_label]]
  print("k label DH pour le num_label_SIO =", k)
  labels_DH_k = labels_DH_df[labels_DH_df['labels_DH'] == k]
  dh_indices_in_k = labels_DH_k.index.tolist()
  indices_en_decallage = list(set(sio_indices_in_num_label) - set(dh_indices_in_k))
  if len(indices_en_decallage) > 0:
    print("Indices en décallage:", indices_en_decallage)
    liste_ref_KO.append(references_SIO.iloc[indices_en_decallage])
print("__________________________________________________")
print ("Longueur anomalie=", len(liste_ref_KO))
for j in range(len(liste_ref_KO)):
  print("Réf ", j, " = ", liste_ref_KO[j])



num_label= 0
-------
k label DH pour le num_label_SIO = 0
num_label= 1
-------
k label DH pour le num_label_SIO = 1
num_label= 2
-------
k label DH pour le num_label_SIO = 2
num_label= 3
-------
k label DH pour le num_label_SIO = 3
num_label= 4
-------


IndexError: list index out of range

In [ ]:
#i0 = indices_in_sio_not_dh[0]
#print("Donnée DH avec indice ", i0)
#print(X.iloc[i0])
#print("Donnée SIO avec indice ", i0)
#print(Y.iloc[i0])
#print("Référence SIO", references_SIO.iloc[i0], ". Référence DH = ", references_DH.iloc[i0])
liste_ref_KO = []
for i in range(10):
  labels_SIO_i = labels_SIO_df[labels_SIO_df['labels_SIO'] == i]
  sio_indices_in_i = labels_SIO_i.index.tolist()
  for j in range(len(sio_indices_in_i)):
    k = labels_DH[sio_indices_in_0[j]]
    labels_DH_k = labels_DH_df[labels_DH_df['labels_DH'] == k]
    dh_indices_in_k = labels_DH_k.index.tolist()
  k = labels_DH[sio_indices_in_0[0]]
  dh_indices_in_i = labels_DH_i.index.tolist()

  indices_in_sio_not_dh_i = list(set(sio_indices_in_i) - set(dh_indices_in_i))
  if len(indices_in_sio_not_dh_i) > 0:
    print("Indices in labels_DH_i but not in labels_SIO_i:", indices_in_sio_not_dh_i)
    liste_ref_KO.append(references_SIO.iloc[indices_in_sio_not_dh_i])
print("__________________________________________________")
print ("Longueur anomalie=", len(liste_ref_KO))
for j in range(len(liste_ref_KO)):
  print("Réf ", j, " = ", liste_ref_KO[j])



NameError: name 'labels_DH_i' is not defined

In [ ]:
# Deux tableaux à une dimension
#array1 = np.array([1, 2, 3])
#array2 = np.array([4, 5, 6]) # Empiler verticalement
#result = np.vstack((array1, array2))
#print(result)
#print(array1.shape)

#print(result.shape)


liste = []
A = np.empty((0, 10000), dtype=int)

for i in range(10000):
  liste = []
  for j in range(10000):
    liste.append(labels_DH[i] -labels_DH[j])
  A = np.vstack([A, liste])


print(A)



KeyboardInterrupt: 

Fin

In [ ]:
RRR






while (i < n_dif - 1):
    i = i + 1
    if (difference[i] != 0):
        print("i=",i)
        print("différence enregistrement ", str(i), " DH=", labels_DH[i], "// SIO=",labels_SIO[i], "// ref=", ref_cli[i,:])
        #print("enr DH=", fichier_DH.loc[i,:])
        #print("enr SIO=",fichier_SIO.loc[i,:])
        nb_ko = nb_ko + 1
        X = np.delete(X,i,0)
        Y = np.delete(Y,i,0)
        ref_cli = np.delete(ref_cli,i,0)
        db = DBSCAN(eps=0.000000000000001,min_samples=min_groupe).fit(X)
        labels_DH = db.labels_
        db2 = DBSCAN(eps=0.00000000000001, min_samples=min_groupe).fit(Y)
        labels_SIO = db2.labels_
        difference = labels_DH - labels_SIO
        n_dif = n_dif - 1


lg = len(difference)
#nb_ano = 0
#for i in range(lg):
#    if difference[i] !=0:
#        nb_ano = nb_ano + 1
print("ANOMALIES = ", nb_ko)
#  3117773703

fin_exec = time.time()
temps = fin_exec - debut
print(f'Temps d\'exécution : {temps:.2}ms')